<a href="https://colab.research.google.com/github/aryan1429/Big-Data/blob/main/Matrix_mul_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
%%writefile matrix_input.txt
A,0,0,1
A,0,1,2
A,0,2,3
A,1,0,4
A,1,1,5
A,1,2,6
B,0,0,7
B,0,1,8
B,1,0,9
B,1,1,10
B,2,0,11
B,2,1,12

Overwriting matrix_input.txt


In [9]:
%%writefile mapper.py
#!/usr/bin/env python3
import sys

for line in sys.stdin:
    line = line.strip()
    parts = line.split(",")
    if len(parts) != 4:
        continue
    matrix, i, j, value = parts
    i, j, value = int(i), int(j), int(value)

    if matrix == "A":
        print(f"{j}\tA,{i},{value}")
    else:  # matrix B
        print(f"{i}\tB,{j},{value}")


Overwriting mapper.py


In [10]:
%%writefile reducer.py
#!/usr/bin/env python3
import sys
from collections import defaultdict

current_key = None
listA = []
listB = []
result = defaultdict(int)

for line in sys.stdin:
    line = line.strip()
    key, val = line.split("\t", 1)

    if current_key and current_key != key:
        for a in listA:
            for b in listB:
                i, a_val = a
                k, b_val = b
                result[(i, k)] += a_val * b_val
        listA, listB = [], []

    current_key = key
    tag, i, value = val.split(",")
    if tag == "A":
        listA.append((int(i), int(value)))
    else:
        listB.append((int(i), int(value)))

# flush last key
for a in listA:
    for b in listB:
        i, a_val = a
        k, b_val = b
        result[(i, k)] += a_val * b_val

# output final result
for (i, k), val in sorted(result.items()):
    print(f"{i},{k}\t{val}")


Overwriting reducer.py


In [12]:
!cat "/content/matrix_input.txt" | python3 "/content/mapper.py" | sort | python3 "/content/reducer.py"

0,0	58
0,1	64
1,0	139
1,1	154
